Importing the necesssary packages

In [126]:
import pandas as pd
import re
from collections import deque
import time
import math

I created two functions, one to store all the nodes and their positions, and another to store the edges between the nodes, along with their corresponding weights. For the edges, I opted for a dictionary, with values for each of its neighbors and their weights. Since this is an undirected graph, If there is an edge between A and B, B is in the adjacency list of A and A is in the adjacency list of B. To store the nodes, I used an array with tuples to store the x and y coordinates of each node. In addition,I created a find path function that will be used to reconstruct the path that was traversed in each of the search algorithms. In the algorithms, the predecessor of each node was kept track of so that once the goal is found, we can get the path that got us there from the start node.

In [127]:
def create_edges(filename):
  file = open(filename, "r")
  edges = dict()
  for each in file:
    current = each.split(',')
    edges.setdefault(current[0],[]).append((current[1], current[2].strip()))
    edges.setdefault(current[1],[]).append((current[0], current[2].strip()))

  file.close()
  #for key in edges:
    #edges[key] = sorted(edges[key], key=lambda x: extract_numeric_part(x[0]))
  return edges

def create_nodes(filename):
  file = open(filename, "r")
  nodes = []
  for line in file:
    current = line.split(',')
    nodes.append((current[0], current[1], current[2].strip()))
  file.close()

  return nodes


def findPath(dictionary, goal):
  path = []
  while goal is not None:
    path.append(goal)
    goal = dictionary[goal]
  return path[::-1]

DFS Implementation:
I chose to implement DFS with a recursive algorithm, passing in the parameters for edges, the current node that we are expanding, the goal node, the path dictionary, the frontier as an array of nodes yet to be expanded, and an array of the nodes that we have expanded. First, I check if the current node we are expanding is the goal, and if it is, return the path. If the current node is not in the dictionary of edges, this mean it has no neighbors (a dead end) so we remove this node from the path and return. Else, for each neighbor in the adjacency list for the current node, we check to see if the neighbor has been explored yet, and if not, we can append it to the frontier and also add the neighbor as a key to the current node in our path dictionary. We then recursively call dfs using this neighbor as the new current node, either until the goal node is found or we hit a dead end. Once this particular "path of neighbors" has all been explored, we can remove this node from the frontier.

In [128]:
def dfs(edges, current, goal, path, frontier, expanded):
  expanded.append(current)
  if current == goal:
    return expanded, findPath(path, goal)
  if current not in edges:
    path.pop()
    return
  for nb, wt in edges[current]:
    if nb not in expanded:
      frontier.append(nb)
      path[nb] = current
      result = dfs(edges, nb, goal, path, frontier, expanded)
      if result is not None:
        return result



BFS Implementation: I implemented BFS with a recursive algorithm. The parameters are the dictionary of edges, the current node that we are expanding, the goal node, the path as a list of the nodes we have expanded thus far, the frontier of nodes that we have yet to expand, and a dictionary to reconstruct the path. I used deque from collections to implement a queue. The current node is the one that has been in the queue the longest and the current node is appended to the path. If the current node is equal to the goal node, then we return the path. Else, for each neighbor of the current node, we add all of these to the frontier if they are not already in the path, i.e. if they have not already been expanded. I also add the neighbor as the key to the current node so we can construct the path later on. Then we pass in the neighbor and call the bfs recursively until it finds the goal node, or the frontier becomes empty.

In [129]:
def bfs(edges, current, goal, expanded, frontier, path):
    current = frontier.popleft()
    expanded.append(current)
    if current == goal:
      return expanded, findPath(path, goal)
    for nb, wt in edges[current]:
      if nb not in expanded and nb not in frontier:
        frontier.append(nb)
        path[nb] = current
    result = bfs(edges, nb, goal, expanded, frontier, path)
    if result is not None:
      return result

    return False


Creating Data Frames to Compare Later

In [130]:
test1 = pd.DataFrame(columns = ['Nodes Expanded', 'Time'])
test2 = pd.DataFrame(columns = ['Nodes Expanded', 'Time'])
test3 = pd.DataFrame(columns = ['Nodes Expanded', 'Time'])

**Test Case 1 - DFS**
I established the dictionary of edges and the array of nodes. I assigned the start node to be the first node in the nodes file and the goal node to be the last node in the node file. The frontier and the expanded array is empty. The path dictionary is instantiated with our start node.

In [131]:
edges = create_edges('TestCase_01_EdgeList.txt')
nodes = create_nodes('TestCase_01_NodeID.csv')

start = nodes[0][0]
goal = nodes[len(nodes)-1][0]
frontier = []
expanded = []
path = {start: None}

t1 = time.time()
expanded, path = (dfs(edges, start,goal, path,frontier, expanded))
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)
test1.loc['DFS'] = [len(expanded), t2 - t1]





['N_0', 'N_1', 'N_6', 'N_5', 'N_10', 'N_11', 'N_16', 'N_15', 'N_20', 'N_21', 'N_7', 'N_12', 'N_13', 'N_14', 'N_8', 'N_9', 'N_4', 'N_18', 'N_19', 'N_24']
The number of nodes expanded from the frontier is: 20
Time: 0.00015878677368164062


**Test Case 1 - BFS**
After establishing the dictionary of edges and the array of nodes, I instantiate the start variable with the first node in the array of nodes, and the goal node as the last node in the array of nodes. I created the frontier as a queue and added the start node to the frontier, created an array to keep track of the nodes expanded, as well as instantiating a dictionary to keep track of the path.

In [132]:
edges = create_edges('TestCase_01_EdgeList.txt')
nodes = create_nodes('TestCase_01_NodeID.csv')

start = nodes[0][0]
goal = nodes[len(nodes)-1][0]
frontier = deque()
frontier.append(start)
path = {start: None}
expanded = []

t1 = time.time()
expanded, path = (bfs(edges, start, goal, expanded, frontier, path))
print(expanded)
t2 = time.time()
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test1.loc['BFS'] = [len(expanded), t2 - t1]



['N_0', 'N_1', 'N_6', 'N_2', 'N_5', 'N_7', 'N_3', 'N_10', 'N_12', 'N_11', 'N_15', 'N_13', 'N_17', 'N_16', 'N_20', 'N_14', 'N_8', 'N_18', 'N_22', 'N_21', 'N_9', 'N_19', 'N_23', 'N_4', 'N_24']
The number of nodes expanded from the frontier is: 25
Time: 0.002313375473022461


**Test Case 2 - DFS**

In [133]:
edges = create_edges('TestCase_02_EdgeList.txt')
nodes = create_nodes('TestCase_02_NodeID.csv')

start = nodes[0][0]
goal = nodes[len(nodes)-1][0]
frontier = []
expanded = []
path = {start: None}

t1 = time.time()
expanded, path = (dfs(edges, start,goal,path, frontier, expanded))
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test2.loc['DFS'] = [len(expanded), t2 - t1]

['N_0', 'N_10', 'N_20', 'N_11', 'N_1', 'N_2', 'N_3', 'N_4', 'N_13', 'N_14', 'N_23', 'N_33', 'N_24', 'N_34', 'N_35', 'N_45', 'N_55', 'N_44', 'N_54', 'N_64', 'N_43', 'N_53', 'N_63', 'N_42', 'N_32', 'N_31', 'N_41', 'N_51', 'N_50', 'N_61', 'N_60', 'N_71', 'N_81', 'N_91', 'N_90', 'N_80', 'N_70', 'N_72', 'N_82', 'N_62', 'N_30', 'N_40', 'N_21', 'N_52', 'N_36', 'N_46', 'N_56', 'N_66', 'N_57', 'N_67', 'N_68', 'N_69', 'N_78', 'N_79', 'N_89', 'N_99']
The number of nodes expanded from the frontier is: 56
Time: 0.0003788471221923828


**Test Case 2 - BFS**

In [134]:
edges = create_edges('TestCase_02_EdgeList.txt')
nodes = create_nodes('TestCase_02_NodeID.csv')

start = nodes[0][0]
goal = nodes[len(nodes)-1][0]
frontier = deque()
frontier.append(start)
path = {start: None}
expanded = []

t1 = time.time()
expanded, path = (bfs(edges, start, goal, expanded, frontier, path))
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)
test2.loc['BFS'] = [len(expanded), t2 - t1]

['N_0', 'N_10', 'N_20', 'N_11', 'N_1', 'N_2', 'N_3', 'N_12', 'N_4', 'N_13', 'N_14', 'N_23', 'N_33', 'N_24', 'N_22', 'N_34', 'N_25', 'N_35', 'N_15', 'N_26', 'N_45', 'N_36', 'N_16', 'N_55', 'N_44', 'N_46', 'N_37', 'N_6', 'N_54', 'N_43', 'N_56', 'N_47', 'N_27', 'N_38', 'N_5', 'N_64', 'N_53', 'N_42', 'N_66', 'N_57', 'N_28', 'N_17', 'N_48', 'N_63', 'N_32', 'N_52', 'N_67', 'N_58', 'N_18', 'N_31', 'N_68', 'N_59', 'N_8', 'N_41', 'N_30', 'N_21', 'N_69', 'N_78', 'N_49', 'N_7', 'N_9', 'N_51', 'N_40', 'N_79', 'N_77', 'N_19', 'N_50', 'N_61', 'N_89', 'N_87', 'N_29', 'N_60', 'N_71', 'N_99']
The number of nodes expanded from the frontier is: 74
Time: 0.000518798828125


**Test Case 3 - DFS**

In [135]:
edges = create_edges('TestCase_03_EdgeList.txt')
nodes = create_nodes('TestCase_03_NodeID.csv')

start = nodes[0][0]
goal = nodes[len(nodes)-1][0]
frontier = []
expanded = []
path = {start: None}

t1 = time.time()
expanded, path = (dfs(edges, start,goal, path,frontier, expanded))
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)
test3.loc['DFS'] = [len(expanded), t2 - t1]

['N_0', 'N_1', 'N_2', 'N_102', 'N_100', 'N_200', 'N_300', 'N_400', 'N_500', 'N_501', 'N_600', 'N_401', 'N_301', 'N_302', 'N_402', 'N_502', 'N_602', 'N_601', 'N_603', 'N_503', 'N_504', 'N_404', 'N_403', 'N_303', 'N_304', 'N_203', 'N_204', 'N_205', 'N_305', 'N_206', 'N_306', 'N_406', 'N_405', 'N_505', 'N_506', 'N_606', 'N_607', 'N_707', 'N_807', 'N_608', 'N_708', 'N_808', 'N_908', 'N_909', 'N_910', 'N_907', 'N_809', 'N_810', 'N_709', 'N_609', 'N_610', 'N_611', 'N_711', 'N_712', 'N_713', 'N_813', 'N_812', 'N_912', 'N_913', 'N_914', 'N_710', 'N_811', 'N_911', 'N_407', 'N_207', 'N_107', 'N_108', 'N_208', 'N_209', 'N_309', 'N_310', 'N_311', 'N_211', 'N_312', 'N_412', 'N_411', 'N_410', 'N_409', 'N_509', 'N_508', 'N_507', 'N_408', 'N_510', 'N_511', 'N_308', 'N_8', 'N_9', 'N_10', 'N_109', 'N_110', 'N_111', 'N_11', 'N_12', 'N_112', 'N_212', 'N_213', 'N_113', 'N_114', 'N_14', 'N_13', 'N_115', 'N_15', 'N_116', 'N_117', 'N_217', 'N_218', 'N_317', 'N_318', 'N_418', 'N_419', 'N_519', 'N_420', 'N_518'

**Test Case 3 - BFS**


In [136]:
edges = create_edges('TestCase_03_EdgeList.txt')
nodes = create_nodes('TestCase_03_NodeID.csv')

start = nodes[0][0]
goal = nodes[len(nodes)-1][0]
frontier = deque()
frontier.append(start)
path = {start: None}
expanded = []
try:
  t1 = time.time()
  expanded, path = (bfs(edges, start, goal, expanded, frontier, path))
  t2 = time.time()
  print(path)
  print("The number of nodes expanded from the frontier is:", len(expanded))
  print("Time:", t2 - t1)
  test3.loc['BFS'] = [len(expanded), t2 - t1]
except:
  print("reached maximum recursion depth!")
  test3.loc['BFS'] = ['NA', 'NA']

reached maximum recursion depth!


**Preparing Functions for Algorithm**
To apply the heuristic, I wanted to be able to access the nodes in a different way, but I rewrote the edges function just for consistency sake. For the nodes, I opted for a dictionary, that way I can store both the coordinates and the distance function later. Since this was processed different as a dictionary, to be able to determine the start node and goal node, I had to do this differently and wrote separate functions to parse through the file again the access the first line for the start node and the last line for the goal node.

In [137]:
def create_edges1(filename):
  file = open(filename, "r")
  edges = dict()
  for each in file:
    current = each.split(',')
    edges.setdefault(current[0],[]).append((current[1], current[2].strip()))
    edges.setdefault(current[1],[]).append((current[0], current[2].strip()))

  file.close()

  return edges

def create_nodes1(filename):
  file = open(filename, "r")
  nodes = dict()
  for line in file:
    current = line.split(',')
    nodes.setdefault(current[0], []).append((current[1], current[2].strip()))
  file.close()

  return nodes

def get_goal(filename):
    file = open(filename, "r")
    lastline = (list(file)[-1])
    goal = lastline.split(',')[0]
    file.close()
    return goal

def get_start(filename):
    file = open(filename, "r")
    firstline = (list(file)[0])
    start = firstline.split(',')[0]
    file.close()
    return start

**Heuristic 1 - Euclidean Distance** I wrote a function to be able to add the straight line distance of each node to the goal node. I thought this would be useful as it ensures that we will never be choosing a node that is going farther away from the goal node, when there is a better option available. I used the formula, taking the square root of the sum of the squares of the differences between the x and the y coordinates of the current node and the goal node. I then added this computed distance as a value for the node in the dictionary. This is an admissible heuristic, since it calculates the straight line distance from that node to the goal node, this will always be shorter than the actual path (if there is one) from that node to the goal node, since the list of edges will only allow forward, right, left, or back "movements", and the straight line will always be shorter (pythagorean theorem). Since our heuristic will never overestimate the cost, this heuristic is admissible.

In [138]:
def euclidean(nodes, goal):

  goal_x = nodes[goal][0][0]
  goal_y = nodes[goal][0][1]

  for node in nodes:
    path = {node: None}
    expanded, path = dfs(edges, node,goal, path, [], [])
    if path is not None:
      actual = len(path)
    else:
      actual = math.inf
    calculated = math.sqrt(math.pow((int(nodes[node][0][0]) - int(goal_x)),2) + math.pow((int(nodes[node][0][1]) - int(goal_y)),2))
    distance = calculated + actual
    nodes[node].append(distance)
  return nodes

**A* - Heuristic 1**
With the new information of the straightline distance between each node and the goal node. I augmented the frontier, so that it is sorted by the distances, with the node with the shortest distance to be expanded next at every step. This function calls itself recursively with the sorted frontier so that it is always progressively adding to the path the node that is closest to the goal node.

In [139]:
def heuristic1(edges, current, goal, path, frontier, expanded):
    expanded.append(current)
    if current == goal:
      return expanded, findPath(path, goal)
    if current not in edges:
      return
    for nb, wt in edges[current]:
      if nb not in frontier and nb not in path:
        frontier.append(nb)
        path[nb] = current
    frontier.pop(0)
    frontier.sort(key=lambda x: nodes[x][1])

    result = heuristic1(edges, frontier[0], goal, path, frontier, expanded)
    if result is not None:
        return result

    # Backtrack: remove the current node from the path and the frontier
    path.pop()
    if current in frontier:
        frontier.remove(current)


    return False

def findPath(dictionary, goal):
  path = []
  while goal is not None:
    path.append(goal)
    goal = dictionary[goal]
  return path[::-1]

**Test Case 1 - A* Heuristic 1**

In [140]:
edges = create_edges1('TestCase_01_EdgeList.txt')
nodes = create_nodes1('TestCase_01_NodeID.csv')
goal = get_goal('TestCase_01_NodeID.csv')
start = get_start('TestCase_01_NodeID.csv')
frontier = []
frontier.append(start)
euclidean(nodes, goal)
expanded = []
path = {start: None}

t1 = time.time()
expanded, path = heuristic1(edges, start, goal, path, frontier, expanded)
t2 = time.time()

print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test1.loc['Eucliean'] = [len(expanded), t2 - t1]



['N_0', 'N_1', 'N_6', 'N_7', 'N_12', 'N_13', 'N_18', 'N_19', 'N_24']
The number of nodes expanded from the frontier is: 9
Time: 0.00016498565673828125


**Test Case 2 - A* Heuristic 1**

In [141]:
edges = create_edges1('TestCase_02_EdgeList.txt')
nodes = create_nodes1('TestCase_02_NodeID.csv')
goal = get_goal('TestCase_02_NodeID.csv')
start = get_start('TestCase_02_NodeID.csv')
frontier = []
frontier.append(start)
euclidean(nodes, goal)
expanded = []
path = {start: None}
t1 = time.time()
expanded, path = heuristic1(edges, start, goal, path, frontier, expanded)
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test2.loc['Euclidean'] = [len(expanded), t2 - t1]

['N_0', 'N_10', 'N_11', 'N_1', 'N_2', 'N_3', 'N_13', 'N_23', 'N_24', 'N_34', 'N_35', 'N_36', 'N_46', 'N_56', 'N_57', 'N_67', 'N_68', 'N_78', 'N_79', 'N_89', 'N_99']
The number of nodes expanded from the frontier is: 21
Time: 0.00026154518127441406


**Test Case 3 - A* Heuristic 1**

In [142]:
edges = create_edges1('TestCase_03_EdgeList.txt')
nodes = create_nodes1('TestCase_03_NodeID.csv')
goal = get_goal('TestCase_03_NodeID.csv')
start = get_start('TestCase_03_NodeID.csv')
frontier = []
frontier.append(start)
euclidean(nodes, goal)
path = {start: None}
t1 = time.time()
expanded, path = heuristic1(edges, start, goal, path, frontier, expanded)
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test3.loc['Euclidean'] = [len(expanded), t2 - t1]

['N_0', 'N_10', 'N_11', 'N_1', 'N_2', 'N_3', 'N_13', 'N_23', 'N_24', 'N_34', 'N_35', 'N_36', 'N_46', 'N_56', 'N_57', 'N_67', 'N_68', 'N_78', 'N_79', 'N_89', 'N_99', 'N_0', 'N_100', 'N_200', 'N_300', 'N_301', 'N_302', 'N_402', 'N_502', 'N_602', 'N_603', 'N_503', 'N_403', 'N_303', 'N_203', 'N_204', 'N_205', 'N_206', 'N_207', 'N_107', 'N_108', 'N_109', 'N_110', 'N_111', 'N_112', 'N_212', 'N_213', 'N_113', 'N_114', 'N_115', 'N_116', 'N_117', 'N_17', 'N_18', 'N_19', 'N_20', 'N_21', 'N_121', 'N_122', 'N_123', 'N_124', 'N_125', 'N_225', 'N_226', 'N_126', 'N_127', 'N_227', 'N_228', 'N_128', 'N_129', 'N_130', 'N_230', 'N_330', 'N_331', 'N_332', 'N_333', 'N_233', 'N_234', 'N_235', 'N_135', 'N_35', 'N_36', 'N_37', 'N_38', 'N_138', 'N_139', 'N_140', 'N_40', 'N_41', 'N_42', 'N_142', 'N_242', 'N_243', 'N_244', 'N_144', 'N_145', 'N_45', 'N_46', 'N_146', 'N_147', 'N_148', 'N_149', 'N_249', 'N_349', 'N_449', 'N_450', 'N_550', 'N_551', 'N_451', 'N_351', 'N_352', 'N_252', 'N_253', 'N_153', 'N_53', 'N_54'

**Heuristic 2** For my second heuristic to test for my informed search, I chose to use the manhattan distance to calculate the distance between each node and the goal node and add this information to each node.I felt this was useful because this would allow the algorithm to ensure that it was never straying farther away from the goal node. The manhattan distance is calculated as the sum of the absolute value of the corresponding coordinates or two points. Since this calculates the distance using up, down, right, and left movements, this mimics the movement of the grid. The calculated distance may be equal to the actual distance, but it will never be greater than. Thus, since our herustic never overestimates the cost, this is an admissible heuristic.

In [143]:
def manhattan(nodes, goal):

  goal_x = nodes[goal][0][0]
  goal_y = nodes[goal][0][1]
  for node in nodes:
    path = {node: None}
    expanded, path = dfs(edges, node,goal, path, [], [])
    if path is not None:
      actual = len(path)
    else:
      actual = math.inf
    calculated = abs(int(nodes[node][0][0]) - int(goal_x)) + abs(int(nodes[node][0][1]) - int(goal_y))
    distance = calculated + actual
    nodes[node].append(distance)
  return nodes

**Test Case 1 - A* Heuristic 2**

In [144]:
edges = create_edges1('TestCase_01_EdgeList.txt')
nodes = create_nodes1('TestCase_01_NodeID.csv')
goal = get_goal('TestCase_01_NodeID.csv')
start = get_start('TestCase_01_NodeID.csv')
frontier = []
frontier.append(start)
manhattan(nodes, goal)
path = {start: None}
expanded = []

t1 = time.time()
expanded, path = heuristic1(edges, start, goal, path, frontier, expanded)
t2 = time.time()

print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test1.loc['Manhattan'] = [len(expanded), t2 - t1]


['N_0', 'N_1', 'N_6', 'N_7', 'N_12', 'N_13', 'N_18', 'N_19', 'N_24']
The number of nodes expanded from the frontier is: 9
Time: 0.00014519691467285156


**Test Case 2 - A* Heuristic 2**

In [145]:
edges = create_edges1('TestCase_02_EdgeList.txt')
nodes = create_nodes1('TestCase_02_NodeID.csv')
goal = get_goal('TestCase_02_NodeID.csv')
start = get_start('TestCase_02_NodeID.csv')
frontier = []
frontier.append(start)
manhattan(nodes, goal)
path = {start:None}
expanded = []



t1 = time.time()
expanded, path = heuristic1(edges, start, goal, path, frontier, expanded)
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time:", t2 - t1)

test2.loc['Manhattan'] = [len(expanded), t2 - t1]

['N_0', 'N_10', 'N_11', 'N_1', 'N_2', 'N_3', 'N_13', 'N_23', 'N_24', 'N_34', 'N_35', 'N_36', 'N_46', 'N_56', 'N_57', 'N_67', 'N_68', 'N_78', 'N_79', 'N_89', 'N_99']
The number of nodes expanded from the frontier is: 21
Time: 0.00014543533325195312


**Test Case 3 - A* Heuristic 2**

In [146]:
edges = create_edges1('TestCase_03_EdgeList.txt')
nodes = create_nodes1('TestCase_03_NodeID.csv')
goal = get_goal('TestCase_03_NodeID.csv')
start = get_start('TestCase_03_NodeID.csv')
frontier = []
frontier.append(start)
manhattan(nodes, goal)
path = {start:None}
expanded = []


t1 = time.time()
expanded, path = heuristic1(edges, start, goal, path, frontier, expanded)
t2 = time.time()
print(expanded)
print("The number of nodes expanded from the frontier is:", len(expanded))
print("Time", t2 - t1)

test3.loc['Manhattan'] = [len(expanded), t2 - t1]

['N_0', 'N_100', 'N_200', 'N_300', 'N_301', 'N_302', 'N_402', 'N_502', 'N_602', 'N_603', 'N_503', 'N_703', 'N_604', 'N_504', 'N_403', 'N_803', 'N_704', 'N_605', 'N_303', 'N_903', 'N_804', 'N_705', 'N_304', 'N_203', 'N_204', 'N_205', 'N_206', 'N_207', 'N_107', 'N_108', 'N_109', 'N_110', 'N_111', 'N_112', 'N_212', 'N_213', 'N_113', 'N_114', 'N_115', 'N_116', 'N_117', 'N_217', 'N_17', 'N_18', 'N_19', 'N_20', 'N_21', 'N_121', 'N_122', 'N_123', 'N_124', 'N_125', 'N_225', 'N_226', 'N_126', 'N_127', 'N_227', 'N_228', 'N_128', 'N_129', 'N_130', 'N_230', 'N_330', 'N_331', 'N_332', 'N_333', 'N_233', 'N_234', 'N_235', 'N_135', 'N_35', 'N_36', 'N_37', 'N_38', 'N_138', 'N_139', 'N_140', 'N_40', 'N_41', 'N_42', 'N_142', 'N_242', 'N_243', 'N_244', 'N_144', 'N_145', 'N_45', 'N_46', 'N_146', 'N_147', 'N_148', 'N_149', 'N_249', 'N_349', 'N_449', 'N_450', 'N_550', 'N_551', 'N_451', 'N_351', 'N_352', 'N_252', 'N_253', 'N_153', 'N_53', 'N_54', 'N_55', 'N_155', 'N_156', 'N_157', 'N_158', 'N_159', 'N_59', 'N

**Comparisons**

In [147]:
test1

,Nodes Expanded,Time
DFS,20.0,0.000159
BFS,25.0,0.002313
Eucliean,9.0,0.000165
Manhattan,9.0,0.000145


In [148]:
test2

,Nodes Expanded,Time
DFS,56.0,0.000379
BFS,74.0,0.000519
Euclidean,21.0,0.000262
Manhattan,21.0,0.000145


In [149]:
test3

,Nodes Expanded,Time
DFS,424.0,0.006779
BFS,NA,NA
Euclidean,186.0,0.001953
Manhattan,176.0,0.001987


**Findings**

To compare between the different approaches, I calculated the time it took to find the solution, as well as the number of nodes that were expanded to find the path. For Test Case 1, The number of nodes expanded for BFS and DFS were similar (25 and 20 respectively) while the BFS took a slightly longer time to run. The informed search expanded significantly less nodes (9) for both the Euclidean and Manhattan distance heuristics. The time it took for the algorithm to run was similar to that of the DFS. For Test Case 2, the uninformed searches had significantly greater number of nodes expanded, almost triple that of the informed searches. In addition, the BFS took a lot longer to run as well. For Test Case 3, I was unable to get results for BFS as the maximum recursion depth was reached. But given the trend from Test Case 1 to Test Case 2, I would imagine it took even longer than DFS to run, with significantly many more nodes expanded. The informed searches only expanded 1/3 the number of nodes as DFS and they took a lot less time as well. For this problem, the informed searches helped the algorithm be more efficient and is more suitable, because the added information of knowing if a node was closer or farther away from the goal node than you already are, helps eliminate a lot of the nodes in the frontier. Within the uninformed searches however, DFS is a more appropriate choice, especially when the grid is larger and thus, the goal is farther away from the start node.